^fill^README.md^here^

^fill^theory.md^here^

## Demonstration

### 1. Setup

#### 1.1. Library imports

Import libraries needed by the calculation. The external libraries used are:

- [numpy](http://www.numpy.org/)

- [DataModelDict](https://github.com/usnistgov/DataModelDict)

- [atomman](https://github.com/usnistgov/atomman)

- [iprPy](https://github.com/usnistgov/iprPy)

In [ ]:
# Standard library imports
from pathlib import Path
import os
import sys
import uuid
import shutil
import datetime
from copy import deepcopy

# http://www.numpy.org/
import numpy as np  

# https://github.com/usnistgov/DataModelDict 
from DataModelDict import DataModelDict as DM

# https://github.com/usnistgov/atomman 
import atomman as am
import atomman.lammps as lmp
import atomman.unitconvert as uc

# https://github.com/usnistgov/iprPy
import iprPy

print('Notebook last executed on', datetime.date.today(), 'using iprPy version', iprPy.__version__)

#### 1.2. Default calculation setup

In [ ]:
# Specify calculation style
calc_style = 'relax_static'

# If workingdir is already set, then do nothing (already in correct folder)
try:
    workingdir = workingdir

# Change to workingdir if not already there
except:
    workingdir = Path('calculationfiles', calc_style)
    if not workingdir.is_dir():
        workingdir.mkdir(parents=True)
    os.chdir(workingdir)

### 2. Assign values for the calculation's run parameters

#### 2.1. Specify system-specific paths

- __lammps_command__ is the LAMMPS command to use (required).

- __mpi_command__ MPI command for running LAMMPS in parallel. A value of None will run simulations serially.

In [ ]:
lammps_command = 'lmp_serial'
mpi_command = None

#### 2.2. Load interatomic potential

- __potential_name__ gives the name of the potential_LAMMPS reference record in the iprPy library to use for the calculation.  

- __potential_file__ gives the path to the potential_LAMMPS reference record to use.  Here, this parameter is automatically generated using potential_name and librarydir.

- __potential_dir__ gives the path for the folder containing the artifacts associated with the potential (i.e. eam.alloy file).  Here, this parameter is automatically generated using potential_name and librarydir.

- __potential__ is an atomman.lammps.Potential object (required).  Here, this parameter is automatically generated from potential_file and potential_dir.

In [ ]:
potential_name = '1999--Mishin-Y--Ni--LAMMPS--ipr1'

# Define potential_file and potential_dir using librarydir and potential_name
potential_file = Path(iprPy.libdir, 'potential_LAMMPS', f'{potential_name}.json')
potential_dir = Path(iprPy.libdir, 'potential_LAMMPS', potential_name)

# Initialize Potential object using potential_file and potential_dir.
potential = lmp.Potential(potential_file, potential_dir)
print('Successfully loaded potential', potential)

#### 2.3. Load initial unit cell system

- __prototype_name__ gives the name of the crystal_prototype reference record in the iprPy library to load. 

- __symbols__ is a list of the potential's elemental model symbols to associate with the unique atom types of the loaded system. 

- __box_parameters__ is a list of the a, b, c lattice constants to assign to the loaded file.

- __load_file__ gives the path to the atomic configuration file to load for the ucell system.  Here, this is generated automatically using prototype_name and librarydir.

- __load_style__ specifies the format of load_file.  Here, this is automatically set for crystal_prototype records.

- __load_options__ specifies any other keyword options for properly loading the load_file.  Here, this is automatically set for crystal_prototype records.

- __ucell__ is an atomman.System representing a fundamental unit cell of the system (required).  Here, this is generated using the load_* parameters and symbols.

In [ ]:
prototype_name = 'A1--Cu--fcc'
symbols = ['Ni']
box_parameters = uc.set_in_units([3.5, 3.5, 3.5], 'angstrom')

# Define load_file using librarydir and prototype_name
load_file = Path(iprPy.libdir, 'crystal_prototype', f'{prototype_name}.json')

# Define load_style and load_options for crystal_prototype records
load_style = 'system_model'
load_options = {}

# Create ucell by loading prototype record
ucell = am.load(load_style, load_file, symbols=symbols, **load_options)

# Rescale ucell using box_parameters
ucell.box_set(a=box_parameters[0], b=box_parameters[1], c=box_parameters[2], scale=True)

print(ucell)

#### 2.4. Modify system

- __sizemults__ list of three integers specifying how many times the ucell vectors of $a$, $b$ and $c$ are replicated in creating system.

- __system__ is an atomman.System to perform the scan on (required). 

In [ ]:
sizemults = [3, 3, 3]

# Generate system by supersizing ucell
system = ucell.supersize(*sizemults)
print('# of atoms in system =', system.natoms)

#### 2.5. Specify calculation-specific run parameters

- __pressure_xx__ gives the xx component of the pressure to equilibriate the system to.

- __pressure_yy__ gives the yy component of the pressure to equilibriate the system to.

- __pressure_zz__ gives the zz component of the pressure to equilibriate the system to.

- __pressure_xy__ gives the xy component of the pressure to equilibriate the system to.

- __pressure_xz__ gives the xz component of the pressure to equilibriate the system to.

- __pressure_yz__ gives the yz component of the pressure to equilibriate the system to.

- __displacementkick__ specifies a multiplier for a random shift of atomic positions to apply prior to relaxation.  This is in length units.

- __energytolerance__ is the energy tolerance to use during the minimizations. This is unitless.

- __forcetolerance__ is the force tolerance to use during the minimizations. This is in energy/length units.

- __maxiterations__ is the maximum number of minimization iterations to use.

- __maxevaluations__ is the maximum number of minimization evaluations to use.

- __maxatommotion__ is the largest distance that an atom is allowed to move during a minimization iteration. This is in length units.

- __maxcycles__ is the maximum number of minimization runs (cycles) to perform.

- __cycletolerance__ is the relative tolerance to use in identifying if the lattice constants have converged from one cycle to the next. 

In [ ]:
pressure_xx = uc.set_in_units(0.0, 'GPa')
pressure_yy = uc.set_in_units(0.0, 'GPa')
pressure_zz = uc.set_in_units(0.0, 'GPa')
pressure_xy = uc.set_in_units(0.0, 'GPa')
pressure_xz = uc.set_in_units(0.0, 'GPa')
pressure_yz = uc.set_in_units(0.0, 'GPa')
displacementkick = uc.set_in_units(0.00001, 'angstrom')
energytolerance = 1e-8
forcetolerance = uc.set_in_units(0.0, 'eV/angstrom')
maxiterations = 10000
maxevaluations = 100000
maxatommotion = uc.set_in_units(0.01, 'angstrom')
maxcycles = 100
cycletolerance = 1e-7

### 3. Define calculation function(s) and generate template LAMMPS script(s)

#### 3.1. minbox.template

In [ ]:
with open('minbox.template', 'w') as f:
    f.write("""^fill^minbox.template^here^""")

#### 3.2. relax_static()

In [ ]:
^fill^calc_relax_static.py(relax_static)^here^

### 4. Run calculation function(s)

In [ ]:
results_dict = relax_static(lammps_command, system, potential,
                            mpi_command = mpi_command,
                            p_xx = pressure_xx, 
                            p_yy = pressure_yy, 
                            p_zz = pressure_zz,
                            p_xy = pressure_xy, 
                            p_xz = pressure_xz, 
                            p_yz = pressure_yz,                            
                            dispmult = displacementkick,
                            etol = energytolerance,
                            ftol = forcetolerance,
                            maxiter = maxiterations,
                            maxeval = maxevaluations,
                            dmax = maxatommotion,
                            maxcycles = maxcycles,
                            ctol = cycletolerance)

In [ ]:
results_dict.keys()

### 5. Report results

#### 5.1. Define units for outputting values

- __length_unit__ is the unit of length to display values in.
- __energy_unit__ is the unit of energy to display values in.
- __pressure_unit__ is the unit of pressure to display values in.

In [ ]:
length_unit = 'angstrom'
energy_unit = 'eV'
pressure_unit = 'GPa'

#### 5.2. Print Ecoh and lattice constants of relaxed ucell

In [ ]:
print('Ecoh =', uc.get_in_units(results_dict['E_coh'], energy_unit), energy_unit)

box = am.Box(lx=results_dict['lx'], ly=results_dict['ly'], lz=results_dict['lz'],
             xy=results_dict['xy'], xz=results_dict['xz'], yz=results_dict['yz'])

print('a =', uc.get_in_units(box.a / sizemults[0], length_unit), length_unit)
print('b =', uc.get_in_units(box.b / sizemults[1], length_unit), length_unit) 
print('c =', uc.get_in_units(box.c / sizemults[2], length_unit), length_unit) 
print('alpha =', box.alpha)
print('beta = ', box.beta)
print('gamma =', box.gamma)

#### 5.3. Check final system pressures

In [ ]:
print('Pxx =', uc.get_in_units(results_dict['measured_pxx'], pressure_unit), pressure_unit)
print('Pyy =', uc.get_in_units(results_dict['measured_pyy'], pressure_unit), pressure_unit)
print('Pzz =', uc.get_in_units(results_dict['measured_pzz'], pressure_unit), pressure_unit)
print('Pyz =', uc.get_in_units(results_dict['measured_pyz'], pressure_unit), pressure_unit)
print('Pxz =', uc.get_in_units(results_dict['measured_pxz'], pressure_unit), pressure_unit)
print('Pxy =', uc.get_in_units(results_dict['measured_pxy'], pressure_unit), pressure_unit)

#### 5.4. Show relaxed atomic configuration

In [ ]:
finalsystem = am.load('atom_dump', results_dict['dumpfile_final'],
                      symbols=results_dict['symbols_final'])
print(finalsystem)